In [ ]:
import sys
import os
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
from matplotlib.patches import Ellipse

from matplotlib import transforms

import seaborn as sns
import pandas as pd

import scipy
from statsmodels.stats import multitest
from scipy.stats import gaussian_kde


# clustering libraries
import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

import umap

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [ ]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_artist(ellipse)

In [ ]:
reducer = umap.UMAP(random_state=42)

In [ ]:
gliph = pd.read_csv("parsed_gliph_file.tsv", sep="\t")

gliph_data_t = gliph.loc[:,"A02_Yes:3M":].transpose()

#gliph_data_t_scaled = RobustScaler(with_centering=False).fit_transform(gliph_data_t)
#gliph_data_t_scaled = StandardScaler().fit_transform(gliph_data_t)
gliph_data_t_scaled = MaxAbsScaler().fit_transform(gliph_data_t)

gliph_data_t_scaled_df = pd.DataFrame(gliph_data_t_scaled, columns = gliph_data_t.columns, index=gliph_data_t.index.values)


gliph_embedding = reducer.fit_transform(gliph_data_t_scaled)
gliph.shape

In [ ]:
gliph_data = gliph.loc[:,"A02_Yes:3M":]

#gliph_data_scaled = StandardScaler().fit_transform(gliph_data)
#gliph_data_scaled = RobustScaler(with_centering=False).fit_transform(gliph_data)
gliph_data_scaled = MaxAbsScaler().fit_transform(gliph_data)

gliph_data_scaled.shape

gliph_data_scaled_df = pd.DataFrame(gliph_data_scaled, columns = gliph_data.columns)

In [ ]:
gliph_data_t_scaled_df.head()

In [ ]:
gliph.head()

In [ ]:
gliph_data_t.index.values

In [ ]:
response_yesno = list()
timepoints_all = list()
responders = list()
non_responders = list()
timepoints_bl = list()
timepoints_6w = list()
timepoints_3m = list()

for sample in gliph_data_t.index.values:
    splitone = sample.split(":")
    splittwo = splitone[0].split("_")
    timepoints_all.append(splitone[1])
    response_yesno.append(splittwo[1])

    if splitone[1] == "BL":
        timepoints_bl.append(sample)
    if splitone[1] == "6W":
        timepoints_6w.append(sample)
    if splitone[1] == "3M":
        timepoints_3m.append(sample)
    if splittwo[1] == "Yes":
        responders.append(sample)
    if splittwo[1] == "No":
        non_responders.append(sample)

print(response_yesno)
print(timepoints_all)
print(timepoints_bl)
print(timepoints_6w)
print(timepoints_3m)
print(responders)
print(non_responders)

In [ ]:
nn = int(gliph_data_scaled.shape[0]/100)
md = 0.25

fit = umap.UMAP(
    n_neighbors=nn,
    min_dist=md,
    n_components=2,
    metric="euclidean",
    random_state=42        
)
gliph_embedding_clusters = fit.fit_transform(gliph_data_scaled)

In [ ]:
plt.scatter(
    gliph_embedding_clusters[:, 0],
    gliph_embedding_clusters[:, 1],
    c="black",
    s=100,
    alpha=0.03)
plt.gca().set_aspect('equal', 'datalim')
plt.title(f'UMAP of GLIPH2 convergence groups\nn_neighbors={nn} min_dist={md}', fontsize=24)

In [ ]:
hdbscan_labels = hdbscan.HDBSCAN(min_samples=None, min_cluster_size=100).fit_predict(gliph_embedding_clusters)
clustered = (hdbscan_labels >= 0)
plt.scatter(gliph_embedding_clusters[~clustered, 0],
            gliph_embedding_clusters[~clustered, 1],
            color=(0.5, 0.5, 0.5),
            s=100,
            alpha=0.5)
plt.scatter(gliph_embedding_clusters[clustered, 0],
            gliph_embedding_clusters[clustered, 1],
            c=hdbscan_labels[clustered],
            s=100,
            cmap='Spectral',
            alpha=0.5)
plt.gca().set_aspect('equal', 'datalim')
plt.title("UMAP of GLIPH2 convergence groups with HDBSCAN clustering.\nGrey nodes unclustered, clustered points in total: {}%".format(
    100*round(float(len(gliph_embedding_clusters[clustered]))/float(len(gliph_embedding_clusters)), 3)), fontsize=24)

In [ ]:
len(set(hdbscan_labels))-1
#Num clusters in hdbscan, -1 because of the unclustered points (

In [ ]:
wilcox_results = pd.DataFrame({'statistic_count': [], 'pvalue_count': [], 'statistic_scaled': [], 'pvalue_scaled': []})

for i in range(0,len(set(hdbscan_labels))-1):
    tmp_count = gliph_data[hdbscan_labels==i]

    tmp_count_yes = tmp_count[responders]

    tmp_count_no = tmp_count[non_responders]

    statistic_count, pvalue_count = scipy.stats.ranksums(np.concatenate(tmp_count_no.to_numpy()), np.concatenate(tmp_count_yes.to_numpy()))
    #wilcox_results.loc[len(wilcox_results.index)] = [statistic, pvalue]
    #print(statistic, pvalue)
    
    tmp_scaled = gliph_data_scaled_df[hdbscan_labels==i]

    tmp_scaled_yes = tmp_scaled[responders]

    tmp_scaled_no = tmp_scaled[non_responders]
    
    statistic_scaled, pvalue_scaled = scipy.stats.ranksums(np.concatenate(tmp_scaled_no.to_numpy()), np.concatenate(tmp_scaled_yes.to_numpy()))
    wilcox_results.loc[len(wilcox_results.index)] = [statistic_count, pvalue_count, statistic_scaled, pvalue_scaled]
wilcox_results.sort_values(by="statistic_scaled", ascending=False)

In [ ]:
fdr_corrected_pvalues = multitest.fdrcorrection(np.concatenate(wilcox_results[["pvalue_count"]].values), alpha=0.01)
hdbscan_cluster_indices_which_pass_test = np.asarray(wilcox_results[fdr_corrected_pvalues[0]].index)
hdbscan_cluster_indices_which_pass_test

In [ ]:
main_clustered = (hdbscan_labels >= 0)
plt.scatter(gliph_embedding_clusters[:, 0],
            gliph_embedding_clusters[:, 1],
            color=(0.5, 0.5, 0.5),
            s=100,
            alpha=0.1)

x_coords = list()
y_coords = list()
cols = list()

for i in range(len(hdbscan_cluster_indices_which_pass_test)):
    cluster_index_points = (hdbscan_labels == hdbscan_cluster_indices_which_pass_test[i])
    #print([cluster_index]*list(cluster_index_points).count(True))
    x_coords += list(gliph_embedding_clusters[cluster_index_points, 0])
    y_coords += list(gliph_embedding_clusters[cluster_index_points, 1])
    cols += [float(i)]*list(cluster_index_points).count(True)


plt.scatter(x_coords,
            y_coords,
            s=100,
            c=cols,
            cmap='Spectral',
            alpha=0.5)
plt.gca().set_aspect('equal', 'datalim')
plt.title("UMAP of GLIPH2 convergence groups \ncoloured if the cluster has a significantly different \ndistribution of clone counts between responders and non-responders")

In [ ]:
cols_individual_points_pure = list()
cols_individual_points_greater = list()
cols_individual_points_log2_1 = list()
cols_individual_points_log2_2 = list()
cols_individual_points_log2_3 = list()

cols_timepoint_which_is_top = list()
cols_timepoint_pure = list()
cols_timepoint_log2_2 = list()

for i in range(gliph.shape[0]):
    yes_count = np.array(gliph_data_scaled_df[responders].loc[i]).sum() / len(responders)
    no_count = np.array(gliph_data_scaled_df[non_responders].loc[i]).sum() / len(non_responders)

    bl_count = np.array(gliph_data_scaled_df[timepoints_bl].loc[i]).sum() / len(timepoints_bl)
    sixW_count = np.array(gliph_data_scaled_df[timepoints_6w].loc[i]).sum() / len(timepoints_6w)
    threeM_count = np.array(gliph_data_scaled_df[timepoints_3m].loc[i]).sum() / len(timepoints_3m)
    
    no_col = '#EE667759'
    mixed_col = '#4477AA59'
    yes_col = '#22883359'

    bl_col = '#EE667759'
    sixW_col = '#4477AA59'
    threeM_col = '#22883359'
    
    else_col = '#AAAAAA03'
    

    if hdbscan_labels[i] in hdbscan_cluster_indices_which_pass_test:
    
        # pure = all counts are from YES or NO, if any count in both it is MIXED
        if yes_count == 0 and no_count != 0:
            cols_individual_points_pure.append(no_col)
        elif no_count == 0 and yes_count != 0:
            cols_individual_points_pure.append(yes_col)
        else:
            cols_individual_points_pure.append(mixed_col) 
    
        if yes_count < no_count:
            cols_individual_points_greater.append(no_col)
        elif yes_count > no_count:
             cols_individual_points_greater.append(yes_col)
        else:
            cols_individual_points_greater.append(mixed_col)
        
        if yes_count < no_count and yes_count*2 < no_count:
            cols_individual_points_log2_1.append(no_col)
        elif yes_count > no_count and yes_count > no_count*2:
            cols_individual_points_log2_1.append(yes_col)
        else:
            cols_individual_points_log2_1.append(mixed_col)
             
        if yes_count < no_count and yes_count*4 < no_count:
            cols_individual_points_log2_2.append(no_col)
        elif yes_count > no_count and yes_count > no_count*4:
            cols_individual_points_log2_2.append(yes_col)
        else:
            cols_individual_points_log2_2.append(mixed_col)
        
        if yes_count < no_count and yes_count*8 < no_count:
            cols_individual_points_log2_3.append(no_col)
        elif yes_count > no_count and yes_count > no_count*8:
            cols_individual_points_log2_3.append(yes_col)
        else:
            cols_individual_points_log2_3.append(mixed_col)


        # timepoint_counts
        if bl_count > threeM_count and bl_count > sixW_count:
            cols_timepoint_which_is_top.append(bl_col)
        elif threeM_count > bl_count and threeM_count > sixW_count:
            cols_timepoint_which_is_top.append(threeM_col)
        elif sixW_count > threeM_count and sixW_count > bl_count:
            cols_timepoint_which_is_top.append(sixW_col)
        else:
            cols_timepoint_which_is_top.append(else_col)

        if bl_count > threeM_count*4 and bl_count > sixW_count*4:
            cols_timepoint_log2_2.append(bl_col)
        elif threeM_count > bl_count*4 and threeM_count > sixW_count*4:
            cols_timepoint_log2_2.append(threeM_col)
        elif sixW_count > threeM_count*4 and sixW_count > bl_count*4:
            cols_timepoint_log2_2.append(sixW_col)
        else:
            cols_timepoint_log2_2.append(else_col)

        if bl_count > 0 and threeM_count == 0 and sixW_count == 0:
            cols_timepoint_pure.append(bl_col)
        elif threeM_count > 0 and bl_count == 0 and sixW_count == 0:
            cols_timepoint_pure.append(threeM_col)
        elif sixW_count > 0 and threeM_count == 0 and bl_count == 0:
            cols_timepoint_pure.append(sixW_col)
        else:
            cols_timepoint_pure.append(else_col)
        
    else:
        
        cols_individual_points_pure.append(else_col)
        cols_individual_points_greater.append(else_col)
        cols_individual_points_log2_1.append(else_col)
        cols_individual_points_log2_2.append(else_col)
        cols_individual_points_log2_3.append(else_col)
        cols_timepoint_which_is_top.append(else_col)
        cols_timepoint_log2_2.append(else_col)
        cols_timepoint_pure.append(else_col)



In [ ]:
len(cols_timepoint_which_is_top)

In [ ]:

######
#FIG 6 A
######

fig = plt.figure()
ax = fig.add_subplot()

test = ax.scatter(gliph_embedding_clusters[:, 0],
            gliph_embedding_clusters[:, 1],
            s=10,
            c=cols_individual_points_log2_2)

plt.gca().set_aspect('equal', 'datalim')
#plt.title("UMAP of GLIPH2 convergence groups", fontsize=30)

for i in range(len(hdbscan_cluster_indices_which_pass_test)):
    
    # Moving the labels slightly
    x_offset_for_cluster_label = 0.2
    y_offset_for_cluster_label = 0.25
    if int(hdbscan_cluster_indices_which_pass_test[i]) > 9:
        x_offset_for_cluster_label = 0.4

    cluster_index_points = (hdbscan_labels == hdbscan_cluster_indices_which_pass_test[i])
    confidence_ellipse(x=gliph_embedding_clusters[cluster_index_points, 0],
                       y=gliph_embedding_clusters[cluster_index_points, 1], 
                       ax=ax, n_std=3.0, edgecolor="#666666", linewidth=2, linestyle="-.")
    plt.text(np.mean(gliph_embedding_clusters[cluster_index_points, 0]) - x_offset_for_cluster_label, 
            np.mean(gliph_embedding_clusters[cluster_index_points, 1]) - y_offset_for_cluster_label, 
            s=str(hdbscan_cluster_indices_which_pass_test[i]), 
            fontsize=18)

custom_points = [Line2D([], [], marker='.', markersize=20, color='#EE667799', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#4477AA99', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#22883399', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#AAAAAA33', linestyle='None')]

custom_lines = [Line2D([], [], color="#666666", lw=2, ls="-.")]

legend1 = ax.legend(custom_points, ["Higher clone count in non-responders", 
                                    "Mixed clone count", 
                                    "Higher clone count in responders", 
                                    "Non-significant clusters"], 
                    loc="upper right", 
                    fontsize=16
                    )
legend2 = ax.legend(custom_lines, ["3 SD ellipse for each separate cluster"], 
                    loc='lower left', 
                    bbox_to_anchor=(0.565, 0.73),
                    fontsize=16
                    )

ax.add_artist(legend1)
ax.add_artist(legend2)

plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.xlabel("UMAP 1", fontsize=24)
plt.ylabel("UMAP 2", fontsize=24)

try:
    os.remove("blood_UMAP_HDBSCANclusters_ranksumtested_log2_2_withclusterlab.svg")
except OSError:
    pass

try:
    os.remove("blood_UMAP_HDBSCANclusters_ranksumtested_log2_2_withclusterlab.png")
except OSError:
    pass

plt.savefig('blood_UMAP_HDBSCANclusters_ranksumtested_log2_2_withclusterlab.svg')
plt.savefig('blood_UMAP_HDBSCANclusters_ranksumtested_log2_2_withclusterlab.png')


In [ ]:

######
#FIG 6 B
######

fig = plt.figure()
ax = fig.add_subplot()

test = ax.scatter(gliph_embedding_clusters[:, 0],
            gliph_embedding_clusters[:, 1],
            s=10,
            c=cols_timepoint_which_is_top)

plt.gca().set_aspect('equal', 'datalim')
#plt.title("UMAP of GLIPH2 convergence groups", fontsize=30)

for i in range(len(hdbscan_cluster_indices_which_pass_test)):
    
    # Moving the labels slightly
    x_offset_for_cluster_label = 0.2
    y_offset_for_cluster_label = 0.25
    if int(hdbscan_cluster_indices_which_pass_test[i]) > 9:
        x_offset_for_cluster_label = 0.4

    cluster_index_points = (hdbscan_labels == hdbscan_cluster_indices_which_pass_test[i])
    confidence_ellipse(x=gliph_embedding_clusters[cluster_index_points, 0],
                       y=gliph_embedding_clusters[cluster_index_points, 1], 
                       ax=ax, n_std=3.0, edgecolor="#666666", linewidth=2, linestyle="-.")
    plt.text(np.mean(gliph_embedding_clusters[cluster_index_points, 0]) - x_offset_for_cluster_label, 
            np.mean(gliph_embedding_clusters[cluster_index_points, 1]) - y_offset_for_cluster_label, 
            s=str(hdbscan_cluster_indices_which_pass_test[i]), 
            fontsize=18)

custom_points = [Line2D([], [], marker='.', markersize=20, color='#EE667799', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#4477AA99', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#22883399', linestyle='None'),
                 Line2D([], [], marker='.', markersize=20, color='#AAAAAA33', linestyle='None')]

custom_lines = [Line2D([], [], color="#666666", lw=2, ls="-.")]

legend1 = ax.legend(custom_points, ["Baseline","6-weeks", "12-weeks",
                                    "Non-significant clusters"], 
                    loc="upper right", 
                    fontsize=16
                    )
legend2 = ax.legend(custom_lines, ["3 SD ellipse for each separate cluster"], 
                    loc='lower left', 
                    bbox_to_anchor=(0.565, 0.73),
                    fontsize=16
                    )

ax.add_artist(legend1)
ax.add_artist(legend2)

plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.xlabel("UMAP 1", fontsize=24)
plt.ylabel("UMAP 2", fontsize=24)

try:
    os.remove("blood_UMAP_HDBSCANclusters_ranksumtested_timepoint_top_withclusterlab.svg")
except OSError:
    pass
try:
    os.remove("blood_UMAP_HDBSCANclusters_ranksumtested_timepoint_top_withclusterlab.png")
except OSError:
    pass

plt.savefig('blood_UMAP_HDBSCANclusters_ranksumtested_timepoint_top_withclusterlab.svg')
plt.savefig('blood_UMAP_HDBSCANclusters_ranksumtested_timepoint_top_withclusterlab.png')


In [ ]:
cluster_indices = dict()

for i in hdbscan_cluster_indices_which_pass_test:
    #print(i)
    cluster_index_patterns = list()
    tmp_data = gliph[hdbscan_labels==i]

    cluster_indices[i] = tmp_data["cluster_index_pattern"].str.split('_').str[0].values
    #print(cluster_indices[i])

for i in hdbscan_cluster_indices_which_pass_test:
    #print(i)
    cluster_index_patterns = list()
    tmp_data = gliph[hdbscan_labels==i]

    cluster_indices[i] = tmp_data["cluster_index_pattern"].str.split('_').str[0].values
    #print(cluster_indices[i])

    with open("../blood_all_run/blood_all_run_fixedhlatypes_cluster.csv", "r") as gliph_cluster_raw_file, open("umap_cluster_"+str(i)+"_raw_GLIPH2.csv", "w") as cluster_outfile:
        for line in gliph_cluster_raw_file:
            if line != "\n":
                ll = line.split(",")
                #print(ll)
                cluster_index = ll[0]
                if cluster_index in cluster_indices[i]:
                    cluster_outfile.write(line)


In [ ]:
outfile = open("blood_index_patterns_significant_responder_clusters.txt", "w")
outfile.write("cluster_index_pattern\n")
for i in hdbscan_cluster_indices_which_pass_test:
    tmp_count = gliph_data[hdbscan_labels==i]

    tmp_count_yes = tmp_count[responders]
    tmp_count_no = tmp_count[non_responders]
    cluster_values_count_no = np.concatenate(tmp_count_no.to_numpy())
    cluster_values_count_yes = np.concatenate(tmp_count_yes.to_numpy())
    
    tmp_scaled = gliph_data_scaled_df[hdbscan_labels==i]

    tmp_scaled_yes = tmp_scaled[responders]
    tmp_scaled_no = tmp_scaled[non_responders]
    cluster_values_scaled_no = np.concatenate(tmp_scaled_no.to_numpy())
    cluster_values_scaled_yes = np.concatenate(tmp_scaled_yes.to_numpy())
    
    tmp_info_cols = gliph[hdbscan_labels==i]
    
    if wilcox_results.iloc[i][2]<0:
        for l in np.concatenate(np.array(tmp_info_cols[["cluster_index_pattern"]])):
            outfile.write(f"{l}\n")
outfile.close()